## Experiment-06
### Amirreza Fosoul and Bithiah Yuan

In [36]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType, DoubleType
from pyspark.sql.functions import desc
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, col, lit, size
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
import random
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
spark = SparkSession.builder.appName('ex6').getOrCreate()
sc = spark.sparkContext


In [2]:
# Unexplode a column to an array of given datatype
def unExplode(df, groupByColName, collectColName, colType):
    types = {'string': StringType(), 'integer': IntegerType()}
    df_collected = df.groupby(groupByColName).agg(f.concat_ws(", ", f.collect_list(df[collectColName])).alias(collectColName))
    result = df_collected.withColumn(collectColName, split(col(collectColName), ",\s*").cast(ArrayType(types[colType])).alias(collectColName)).orderBy(groupByColName)
    return result

In [3]:
# Read user ratings into Dataframe
user_df = spark.read.option("delimiter", ";").csv('./example0.txt')
#user_df = spark.read.option("delimiter", ";").csv('./example.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))

user_pre = user_df

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Count the number of distinct users
numUsers = user_df.select("userID").distinct().count()

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

userLib = user_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType()))).orderBy("userID")

get_len_udf = udf(lambda x: len(x), IntegerType())

userLib = userLib.withColumn("libSize", get_len_udf(col("paperID")))

userLib.show()

+--------------------+--------------------+-------+
|              userID|             paperID|libSize|
+--------------------+--------------------+-------+
|589b870a611c25fa9...|[12832332, 130547...|      5|
|90f1a3e6fcdbf9bc5...|[115945, 11733005...|      5|
+--------------------+--------------------+-------+



In [4]:

# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./paper0.csv')
#w_df = w_df.select("_c0", "_c01" "_c06", "_c09", "_c13", "_c14")
w_df = w_df.select("_c0", "_c1", "_c6", "_c9")
#w_df = w_df.select(col("_c0").alias("paperID"), col("_c01").alias("type"), col("_c06").alias("pages"), col("_c09").alias("year"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.select(col("_c0").alias("paperID"), col("_c1").alias("type"), col("_c6").alias("pages"), col("_c9").alias("year"))

#w_df.show()

### Exercise 6. 1 (Problem Modeling)

In [53]:
# Create a user hash map dataframe by adding a column with descending row numbers 
# to the pre-explode user dataframe
# user_hash_map = user_pre.withColumn("user_idx", F.monotonically_increasing_id().cast(IntegerType()))

# Select the user indices and paperID
#rating = user_hash_map.select('user_idx', 'paperID')
rating = userLib.select('userID', 'paperID', 'libSize')

# # Split the string if all paperIDs into individual paperIDs and cast the datatype to int
# rating = rating.withColumn("papers", split(col("paperID"), ",\s*").cast(ArrayType(IntegerType())).alias("papers"))
# # Drop the column with the string of paperIDs
# rating = rating.drop("paperID")

# Add a column of 1's called ratings for the rated papers w.r.t to the user index
rating = rating.withColumn("rating", lit(1))

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))

# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))

    # Select n random papers from the unrated papers where n is the 
    # length of the rated papers list of each user
    unrated_papers = random.sample(unrated, len(papers))
            
    return unrated_papers


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
rating = rating.withColumn("unrated", get_unrated(rating.paperID))

#Create a dataframe with the rated papers and their ratings of 1's
rated_df = rating.select("userID", f.explode("paperID").alias("paperID"), "rating", "libSize")

# Create a dataframe with the unrated papers
unrated_df = rating.select("userID", f.explode("unrated").alias("unrated"), "libSize")
# Add a column of ratings of 0's to the unrated papers
unrated_df = unrated_df.withColumn("rating", lit(0))
unrated_df = unrated_df.select("userID", col("unrated").alias("paperID"), "rating", "libSize")

# Union the rated and unrated dataframes and order by the user indices
rating_matrix = rated_df.union(unrated_df).orderBy("userID")

feature_df = rating_matrix.join(w_df, ["paperID"]).orderBy("userID")

window = Window.partitionBy(col("userID"))

feature_df_rated = feature_df.filter(feature_df.rating == 1).groupBy("userID", "paperID", "rating", "pages", "year", "type", "libSize").agg(f.avg("pages").over(window).alias("avgPages")).orderBy("userID")

feature_df_unrated = feature_df.filter(feature_df.rating == 0)
feature_df_unrated = feature_df_unrated.withColumn('avgPages', lit(" ").cast(StringType()))
feature_df_unrated = feature_df_unrated.select("userID", "paperID", "rating", "pages", "year", "type", "libSize", "avgPages")

y = feature_df_rated.select('userID', 'avgPages').distinct()

feature_df_unrated =  feature_df_unrated.drop("avgPages")

x = feature_df_unrated.join(y, 'userID')


model_df = feature_df_rated.union(x).orderBy("userID", "rating")
model_df1 = model_df.withColumn("pages", coalesce(model_df.pages, model_df.avgPages))

model_df1.show()

+--------------------+--------+------+-----+----+-----------+-------+--------+
|              userID| paperID|rating|pages|year|       type|libSize|avgPages|
+--------------------+--------+------+-----+----+-----------+-------+--------+
|589b870a611c25fa9...|11733005|     0|    7|2012|    article|      5|    17.0|
|589b870a611c25fa9...| 9045137|     0|   24|2007|    article|      5|    17.0|
|589b870a611c25fa9...|  115945|     0|   11|2005|    article|      5|    17.0|
|589b870a611c25fa9...| 8310458|     0|   15|2008|    article|      5|    17.0|
|589b870a611c25fa9...| 3728173|     0|    5|2007|    article|      5|    17.0|
|589b870a611c25fa9...| 1001231|     1| 17.0|2005|proceedings|      5|    17.0|
|589b870a611c25fa9...|12832332|     1|   29|2013|     inbook|      5|    17.0|
|589b870a611c25fa9...|  352713|     1|   12|2002|       misc|      5|    17.0|
|589b870a611c25fa9...| 1305474|     1| 17.0|2002|       misc|      5|    17.0|
|589b870a611c25fa9...|  956315|     1|   10|2006|   

In [54]:
stringIndexer = StringIndexer(inputCol="type", outputCol="typeIndex")
model = stringIndexer.fit(model_df1)
indexed = model.transform(model_df1)

encoder = OneHotEncoder(inputCol="typeIndex", outputCol="encodedType")
encoded = encoder.transform(indexed)
encoded.show()

+--------------------+--------+------+-----+----+-----------+-------+--------+---------+-------------+
|              userID| paperID|rating|pages|year|       type|libSize|avgPages|typeIndex|  encodedType|
+--------------------+--------+------+-----+----+-----------+-------+--------+---------+-------------+
|589b870a611c25fa9...| 8310458|     0|   15|2008|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 9045137|     0|   24|2007|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 3728173|     0|    5|2007|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...|  115945|     0|   11|2005|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...|11733005|     0|    7|2012|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 1001231|     1| 17.0|2005|proceedings|      5|    17.0|      2.0|(3,[2],[1.0])|
|589b870a611c25fa9...|  352713|     1|   12|2002|       misc|      5|    

In [55]:
model_df1 = encoded.withColumn("pages", encoded["pages"].cast(DoubleType()))
model_df1 = model_df1.withColumn("year", model_df1["year"].cast(IntegerType()))

In [70]:
assembler = VectorAssembler(inputCols=["pages", "year", "libSize", "avgPages", "encodedType"], outputCol="features")

preprocessed_df = assembler.transform(model_df1)
preprocessed_df = preprocessed_df.select("userID", "paperID", col("rating").alias("label"),"features")

preprocessed_df.show()

+--------------------+--------+-----+--------------------+
|              userID| paperID|label|            features|
+--------------------+--------+-----+--------------------+
|589b870a611c25fa9...| 8310458|    0|[15.0,2008.0,5.0,...|
|589b870a611c25fa9...|11733005|    0|[7.0,2012.0,5.0,1...|
|589b870a611c25fa9...| 3728173|    0|[5.0,2007.0,5.0,1...|
|589b870a611c25fa9...| 9045137|    0|[24.0,2007.0,5.0,...|
|589b870a611c25fa9...|  115945|    0|[11.0,2005.0,5.0,...|
|589b870a611c25fa9...|12832332|    1|[29.0,2013.0,5.0,...|
|589b870a611c25fa9...| 1305474|    1|[17.0,2002.0,5.0,...|
|589b870a611c25fa9...| 1001231|    1|[17.0,2005.0,5.0,...|
|589b870a611c25fa9...|  352713|    1|[12.0,2002.0,5.0,...|
|589b870a611c25fa9...|  956315|    1|[10.0,2006.0,5.0,...|
|90f1a3e6fcdbf9bc5...|  956315|    0|[10.0,2006.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1305474|    0|[12.4,2002.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1001231|    0|[12.4,2005.0,5.0,...|
|90f1a3e6fcdbf9bc5...|12832332|    0|[29.0,2013.0,5.0,..

### Exercise 6. 3 (Classificatoin Algorithms)

In [80]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [79]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(preprocessed_df)

#Automatically identify categorical features, and index them.
#We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(preprocessed_df)

# Split the data into training and test sets
(trainingData, testData) = preprocessed_df.randomSplit([0.8, 0.2])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show()

# Select (prediction, true label) and compute test error
evaluator =  RegressionEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         0.0|[7.0,2012.0,5.0,1...|
|       0.0|         0.0|[12.4,2005.0,5.0,...|
|       0.0|         1.0|[5.0,2007.0,5.0,1...|
+----------+------------+--------------------+

Root-mean-square error = 0.8164965809277261


In [82]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(preprocessed_df)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(preprocessed_df)

# Split the data into training and test sets
(trainingData, testData) = preprocessed_df.randomSplit([0.8, 0.2])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show()

# # Select (prediction, true label) and compute test error
# evaluator = MulticlassClassificationEvaluator(
#     labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print("Test Error = %g" % (1.0 - accuracy))

# rfModel = model.stages[2]
# print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|             1|    0|[24.0,2007.0,5.0,...|
|             0|    1|[10.0,2006.0,5.0,...|
|             1|    0|[29.0,2013.0,5.0,...|
+--------------+-----+--------------------+

